# Imports and Setup

In [ ]:
# Core libraries
import os
import pandas as pd
import numpy as np
from typing import Dict, List, Any, Optional, TypedDict
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# LangChain and LangGraph imports
from langchain_aws import ChatBedrock
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import BedrockEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

# Additional utilities
import json
import pickle
from pathlib import Path

print("✅ All imports successful!")

# Configuration and AWS Setup

In [ ]:
# AWS Configuration
AWS_CONFIG = {
    "region_name": "us-west-2",
    "model_id": "anthropic.claude-3-5-sonnet-20241022-v2:0"
}

# Initialize Bedrock client
try:
    bedrock_llm = ChatBedrock(
        model_id=AWS_CONFIG["model_id"],
        region_name=AWS_CONFIG["region_name"],
        model_kwargs={
            "max_tokens": 4000,
            "temperature": 0.1
        }
    )
    
    bedrock_embeddings = BedrockEmbeddings(
        model_id="amazon.titan-embed-text-v1",
        region_name=AWS_CONFIG["region_name"]
    )
    
    print("✅ AWS Bedrock client initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing Bedrock: {e}")

# State Definition

In [ ]:
class AgentState(TypedDict):
    """State for the agent workflow"""
    messages: List[str]
    user_input: str
    current_step: str
    tool_calls: List[str]
    retrieved_info: Dict[str, Any]
    final_response: str
    needs_more_tools: bool
    conversation_history: List[Dict[str, str]]

# Data Loading and Vector Database Creation

In [ ]:
class VectorDatabaseManager:
    """Manages FAISS vector databases for different retrievers"""
    
    def __init__(self, embeddings, persist_directory="./vector_stores"):
        self.embeddings = embeddings
        self.persist_directory = Path(persist_directory)
        self.persist_directory.mkdir(exist_ok=True)
        self.vector_stores = {}
        self.retrievers = {}
        
    def create_vector_store_from_excel(self, excel_path: str, store_name: str, 
                                     text_columns: List[str], metadata_columns: List[str] = None):
        """Create FAISS vector store from Excel file"""
        try:
            # Read Excel file
            df = pd.read_excel(excel_path)
            print(f"📊 Loaded {len(df)} records from {excel_path}")
            
            # Create documents
            documents = []
            for idx, row in df.iterrows():
                # Combine text columns
                text_content = " | ".join([str(row[col]) for col in text_columns if pd.notna(row[col])])
                
                # Create metadata
                metadata = {"source": excel_path, "row_id": idx}
                if metadata_columns:
                    for col in metadata_columns:
                        if col in df.columns and pd.notna(row[col]):
                            metadata[col] = str(row[col])
                
                documents.append(Document(page_content=text_content, metadata=metadata))
            
            # Split documents if needed
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            split_docs = text_splitter.split_documents(documents)
            
            # Create FAISS vector store
            vector_store = FAISS.from_documents(split_docs, self.embeddings)
            
            # Save to disk
            store_path = self.persist_directory / store_name
            vector_store.save_local(str(store_path))
            
            self.vector_stores[store_name] = vector_store
            print(f"✅ Created and saved vector store '{store_name}' with {len(split_docs)} documents")
            
            return vector_store
            
        except Exception as e:
            print(f"❌ Error creating vector store {store_name}: {e}")
            return None
    
    def load_vector_store(self, store_name: str):
        """Load existing vector store from disk"""
        try:
            store_path = self.persist_directory / store_name
            if store_path.exists():
                vector_store = FAISS.load_local(str(store_path), self.embeddings, allow_dangerous_deserialization=True)
                self.vector_stores[store_name] = vector_store
                print(f"✅ Loaded existing vector store: {store_name}")
                return vector_store
            else:
                print(f"⚠️ Vector store {store_name} not found at {store_path}")
                return None
        except Exception as e:
            print(f"❌ Error loading vector store {store_name}: {e}")
            return None
    
    def create_self_query_retriever(self, store_name: str, document_content_description: str, 
                                  metadata_field_info: List[AttributeInfo]):
        """Create self-query retriever for a vector store"""
        try:
            if store_name not in self.vector_stores:
                print(f"❌ Vector store {store_name} not found")
                return None
            
            retriever = SelfQueryRetriever.from_llm(
                llm=bedrock_llm,
                vectorstore=self.vector_stores[store_name],
                document_contents=document_content_description,
                metadata_field_info=metadata_field_info,
                verbose=True
            )
            
            self.retrievers[store_name] = retriever
            print(f"✅ Created self-query retriever for {store_name}")
            return retriever
            
        except Exception as e:
            print(f"❌ Error creating self-query retriever for {store_name}: {e}")
            # Fallback to regular retriever
            retriever = self.vector_stores[store_name].as_retriever(search_kwargs={"k": 5})
            self.retrievers[store_name] = retriever
            print(f"✅ Created fallback retriever for {store_name}")
            return retriever

# Initialize Vector Database Manager
vector_db_manager = VectorDatabaseManager(bedrock_embeddings)

# Sample Data Creation (Replace with your Excel files)

In [ ]:
# Create sample Excel files for demonstration
# Replace this section with your actual Excel file paths

def create_sample_data():
    """Create sample Excel files for testing"""
    
    # Sample Change Management data
    change_data = {
        'change_id': ['CHG001', 'CHG002', 'CHG003'],
        'description': ['Server upgrade maintenance', 'Database migration', 'Network configuration update'],
        'status': ['Completed', 'In Progress', 'Planned'],
        'priority': ['High', 'Medium', 'Low'],
        'assigned_to': ['John Doe', 'Jane Smith', 'Bob Johnson']
    }
    pd.DataFrame(change_data).to_excel('./sample_change_data.xlsx', index=False)
    
    # Sample CMDB data
    cmdb_data = {
        'ci_id': ['CI001', 'CI002', 'CI003'],
        'ci_name': ['Production Server 1', 'Database Server', 'Load Balancer'],
        'ci_type': ['Server', 'Database', 'Network Device'],
        'status': ['Active', 'Active', 'Maintenance'],
        'location': ['Data Center A', 'Data Center B', 'Data Center A']
    }
    pd.DataFrame(cmdb_data).to_excel('./sample_cmdb_data.xlsx', index=False)
    
    # Sample Incident data
    incident_data = {
        'incident_id': ['INC001', 'INC002', 'INC003'],
        'description': ['Server down', 'Database connection issues', 'Network latency problems'],
        'status': ['Resolved', 'Open', 'In Progress'],
        'priority': ['Critical', 'High', 'Medium'],
        'assigned_to': ['Support Team A', 'DBA Team', 'Network Team']
    }
    pd.DataFrame(incident_data).to_excel('./sample_incident_data.xlsx', index=False)
    
    print("✅ Sample data files created!")

# Create sample data (remove this if you have actual Excel files)
create_sample_data()

# Vector Store Setup

In [ ]:
# Create or load vector stores for each retriever
print("🔧 Setting up vector stores...")

# Change Management Vector Store
change_vs = vector_db_manager.load_vector_store("change_management")
if not change_vs:
    change_vs = vector_db_manager.create_vector_store_from_excel(
        "./sample_change_data.xlsx",
        "change_management",
        text_columns=['description', 'status', 'priority'],
        metadata_columns=['change_id', 'assigned_to']
    )

# CMDB Vector Store
cmdb_vs = vector_db_manager.load_vector_store("cmdb")
if not cmdb_vs:
    cmdb_vs = vector_db_manager.create_vector_store_from_excel(
        "./sample_cmdb_data.xlsx",
        "cmdb",
        text_columns=['ci_name', 'ci_type', 'status'],
        metadata_columns=['ci_id', 'location']
    )

# Incident Vector Store
incident_vs = vector_db_manager.load_vector_store("incident")
if not incident_vs:
    incident_vs = vector_db_manager.create_vector_store_from_excel(
        "./sample_incident_data.xlsx",
        "incident",
        text_columns=['description', 'status', 'priority'],
        metadata_columns=['incident_id', 'assigned_to']
    )

print("✅ All vector stores ready!")


# Self-Query Retrievers Setup

In [ ]:
# Define metadata field information for self-query retrievers

# Change Management metadata
change_metadata_fields = [
    AttributeInfo(name="change_id", description="Unique change identifier", type="string"),
    AttributeInfo(name="assigned_to", description="Person assigned to the change", type="string"),
    AttributeInfo(name="source", description="Source of the document", type="string")
]

# CMDB metadata
cmdb_metadata_fields = [
    AttributeInfo(name="ci_id", description="Configuration Item identifier", type="string"),
    AttributeInfo(name="location", description="Physical location of the CI", type="string"),
    AttributeInfo(name="source", description="Source of the document", type="string")
]

# Incident metadata
incident_metadata_fields = [
    AttributeInfo(name="incident_id", description="Unique incident identifier", type="string"),
    AttributeInfo(name="assigned_to", description="Team assigned to the incident", type="string"),
    AttributeInfo(name="source", description="Source of the document", type="string")
]

# Create self-query retrievers
change_retriever = vector_db_manager.create_self_query_retriever(
    "change_management",
    "Change management records including descriptions, status, and priority information",
    change_metadata_fields
)

cmdb_retriever = vector_db_manager.create_self_query_retriever(
    "cmdb",
    "Configuration Management Database records with CI information",
    cmdb_metadata_fields
)

incident_retriever = vector_db_manager.create_self_query_retriever(
    "incident",
    "Incident management records with descriptions and status information",
    incident_metadata_fields
)

print("✅ Self-query retrievers created!")

# Tool Definitions

In [ ]:
class ChatbotTools:
    """Collection of tools for the chatbot"""
    
    def __init__(self, llm, retrievers):
        self.llm = llm
        self.retrievers = retrievers
        self.available_prompts = {
            "change_queries": "Ask about change management, deployments, or maintenance activities",
            "cmdb_queries": "Ask about configuration items, servers, databases, or infrastructure",
            "incident_queries": "Ask about incidents, outages, or technical issues",
            "general_questions": "Ask general questions for AI assistance"
        }
    
    def greeting_tool(self, state: AgentState) -> AgentState:
        """Handle greetings and show available prompts"""
        greeting_response = f"""
        Hello! 👋 Welcome to the IT Service Management Assistant!
        
        I can help you with the following:
        
        🔧 **Change Management**: {self.available_prompts['change_queries']}
        🖥️  **CMDB Queries**: {self.available_prompts['cmdb_queries']}
        🚨 **Incident Management**: {self.available_prompts['incident_queries']}
        🤖 **General Questions**: {self.available_prompts['general_questions']}
        
        What would you like to know about today?
        """
        
        state["retrieved_info"]["greeting"] = greeting_response
        state["tool_calls"].append("greeting")
        return state
    
    def change_retriever_tool(self, state: AgentState) -> AgentState:
        """Retrieve change management information"""
        try:
            query = state["user_input"]
            docs = self.retrievers["change_management"].invoke(query)
            
            change_info = []
            for doc in docs:
                change_info.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })
            
            # Generate response based on retrieved information
            context = "\n".join([doc["content"] for doc in change_info])
            response = self.llm.invoke(f"""
            Based on the following change management information, please provide a helpful response to the user query: "{query}"
            
            Change Management Data:
            {context}
            
            Please provide a clear and informative response.
            """).content
            
            state["retrieved_info"]["change_management"] = {
                "response": response,
                "docs": change_info
            }
            state["tool_calls"].append("change_retriever")
            
        except Exception as e:
            state["retrieved_info"]["change_management"] = {
                "response": f"Sorry, I encountered an error retrieving change management information: {str(e)}",
                "docs": []
            }
            state["tool_calls"].append("change_retriever_error")
        
        return state
    
    def cmdb_retriever_tool(self, state: AgentState) -> AgentState:
        """Retrieve CMDB information"""
        try:
            query = state["user_input"]
            docs = self.retrievers["cmdb"].invoke(query)
            
            cmdb_info = []
            for doc in docs:
                cmdb_info.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })
            
            context = "\n".join([doc["content"] for doc in cmdb_info])
            response = self.llm.invoke(f"""
            Based on the following CMDB information, please provide a helpful response to the user query: "{query}"
            
            CMDB Data:
            {context}
            
            Please provide a clear and informative response about the configuration items.
            """).content
            
            state["retrieved_info"]["cmdb"] = {
                "response": response,
                "docs": cmdb_info
            }
            state["tool_calls"].append("cmdb_retriever")
            
        except Exception as e:
            state["retrieved_info"]["cmdb"] = {
                "response": f"Sorry, I encountered an error retrieving CMDB information: {str(e)}",
                "docs": []
            }
            state["tool_calls"].append("cmdb_retriever_error")
        
        return state
    
    def incident_retriever_tool(self, state: AgentState) -> AgentState:
        """Retrieve incident information"""
        try:
            query = state["user_input"]
            docs = self.retrievers["incident"].invoke(query)
            
            incident_info = []
            for doc in docs:
                incident_info.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })
            
            context = "\n".join([doc["content"] for doc in incident_info])
            response = self.llm.invoke(f"""
            Based on the following incident information, please provide a helpful response to the user query: "{query}"
            
            Incident Data:
            {context}
            
            Please provide a clear and informative response about the incidents.
            """).content
            
            state["retrieved_info"]["incidents"] = {
                "response": response,
                "docs": incident_info
            }
            state["tool_calls"].append("incident_retriever")
            
        except Exception as e:
            state["retrieved_info"]["incidents"] = {
                "response": f"Sorry, I encountered an error retrieving incident information: {str(e)}",
                "docs": []
            }
            state["tool_calls"].append("incident_retriever_error")
        
        return state
    
    def basic_llm_tool(self, state: AgentState) -> AgentState:
        """Handle general queries with basic LLM response"""
        try:
            query = state["user_input"]
            response = self.llm.invoke(f"""
            Please provide a helpful response to this general question: "{query}"
            
            Keep your response informative and friendly. If this seems like it should be handled by a specific IT service management tool, 
            suggest that the user ask about change management, CMDB, or incident management specifically.
            """).content
            
            state["retrieved_info"]["general"] = response
            state["tool_calls"].append("basic_llm")
            
        except Exception as e:
            state["retrieved_info"]["general"] = f"Sorry, I encountered an error: {str(e)}"
            state["tool_calls"].append("basic_llm_error")
        
        return state

# Initialize tools
retrievers_dict = {
    "change_management": change_retriever,
    "cmdb": cmdb_retriever,
    "incident": incident_retriever
}

chatbot_tools = ChatbotTools(bedrock_llm, retrievers_dict)
print("✅ Chatbot tools initialized!")

# Router Implementation

In [ ]:
def router_node(state: AgentState) -> AgentState:
    """Router to classify user input and determine which tools to call"""
    
    user_input = state["user_input"].lower()
    state["current_step"] = "routing"
    
    # Classification prompt
    classification_prompt = f"""
    Classify the following user input and determine which tools should be called. 
    The user input is: "{state['user_input']}"
    
    Available tools:
    1. greeting - for greetings like "hi", "hello", "help"
    2. change_retriever - for change management, deployment, maintenance queries
    3. cmdb_retriever - for configuration items, infrastructure, server queries  
    4. incident_retriever - for incident, outage, problem queries
    5. basic_llm - for general questions not related to specific IT services
    
    You can select multiple tools if the query is complex and needs information from multiple sources.
    
    Respond with a JSON object with the following structure:
    {{
        "tools_needed": ["tool1", "tool2", ...],
        "reasoning": "explanation of why these tools were selected"
    }}
    
    Examples:
    - "Hello" -> {{"tools_needed": ["greeting"], "reasoning": "User is greeting"}}
    - "What changes are in progress?" -> {{"tools_needed": ["change_retriever"], "reasoning": "User asking about change management"}}
    - "Show me server CI001 and any related incidents" -> {{"tools_needed": ["cmdb_retriever", "incident_retriever"], "reasoning": "User needs both CMDB info and incident information"}}
    """
    
    try:
        classification_response = bedrock_llm.invoke(classification_prompt).content
        
        # Try to parse JSON response
        try:
            classification_data = json.loads(classification_response)
            tools_needed = classification_data.get("tools_needed", ["basic_llm"])
        except json.JSONDecodeError:
            # Fallback classification based on keywords
            tools_needed = []
            
            if any(greeting in user_input for greeting in ["hi", "hello", "help", "greeting"]):
                tools_needed.append("greeting")
            elif any(word in user_input for word in ["change", "deployment", "maintenance", "chg"]):
                tools_needed.append("change_retriever")
            elif any(word in user_input for word in ["cmdb", "configuration", "server", "ci", "infrastructure"]):
                tools_needed.append("cmdb_retriever")
            elif any(word in user_input for word in ["incident", "outage", "problem", "inc", "issue"]):
                tools_needed.append("incident_retriever")
            else:
                tools_needed.append("basic_llm")
        
        state["messages"].append(f"Router selected tools: {tools_needed}")
        
        # Execute selected tools
        for tool in tools_needed:
            if tool == "greeting":
                state = chatbot_tools.greeting_tool(state)
            elif tool == "change_retriever":
                state = chatbot_tools.change_retriever_tool(state)
            elif tool == "cmdb_retriever":
                state = chatbot_tools.cmdb_retriever_tool(state)
            elif tool == "incident_retriever":
                state = chatbot_tools.incident_retriever_tool(state)
            elif tool == "basic_llm":
                state = chatbot_tools.basic_llm_tool(state)
        
        state["current_step"] = "tool_execution_complete"
        
    except Exception as e:
        state["messages"].append(f"Router error: {str(e)}")
        state["current_step"] = "router_error"
        # Fallback to basic LLM
        state = chatbot_tools.basic_llm_tool(state)
    
    return state

print("✅ Router implemented!")

# Judge/Validator Implementation

In [ ]:
def judge_node(state: AgentState) -> AgentState:
    """Judge to validate if the response is adequate or needs more tools"""
    
    state["current_step"] = "judging"
    
    # Prepare context from all retrieved information
    context_parts = []
    for key, value in state["retrieved_info"].items():
        if isinstance(value, dict) and "response" in value:
            context_parts.append(f"{key}: {value['response']}")
        elif isinstance(value, str):
            context_parts.append(f"{key}: {value}")
    
    combined_context = "\n".join(context_parts)
    
    # Judge prompt
    judge_prompt = f"""
    Evaluate if the following information adequately answers the user's question: "{state['user_input']}"
    
    Retrieved Information:
    {combined_context}
    
    Tools called: {state['tool_calls']}
    
    Please evaluate:
    1. Does the information adequately answer the user's question?
    2. Are there any gaps that would require additional tool calls?
    3. Is the information relevant and helpful?
    
    Respond with a JSON object:
    {{
        "is_adequate": true/false,
        "reasoning": "explanation",
        "suggested_additional_tools": ["tool1", "tool2"] or [],
        "confidence_score": 0.0-1.0
    }}
    """
    
    try:
        judge_response = bedrock_llm.invoke(judge_prompt).content
        
        try:
            judge_data = json.loads(judge_response)
            is_adequate = judge_data.get("is_adequate", True)
            confidence = judge_data.get("confidence_score", 0.8)
        except json.JSONDecodeError:
            # Fallback: assume adequate if we have some retrieved info
            is_adequate = len(state["retrieved_info"]) > 0
            confidence = 0.7
        
        if is_adequate and confidence > 0.6:
            state["needs_more_tools"] = False
            state["current_step"] = "creating_final_response"
        else:
            state["needs_more_tools"] = True
            state["current_step"] = "needs_improvement"
            
    except Exception as e:
        state["messages"].append(f"Judge error: {str(e)}")
        state["needs_more_tools"] = False
        state["current_step"] = "judge_error"
    
    return state

print("✅ Judge/Validator implemented!")

# Response Generation

In [ ]:
def create_final_response(state: AgentState) -> AgentState:
    """Create the final consolidated response"""
    
    state["current_step"] = "generating_response"
    
    if state["needs_more_tools"]:
        # If judge determined response is inadequate
        fallback_response = """
        I've gathered some information for you, but I feel it might not be complete. 
        For now, here's what I found. We are continuously working on improving the system to provide better responses.
        
        """
        
        # Add available information
        info_parts = []
        for key, value in state["retrieved_info"].items():
            if isinstance(value, dict) and "response" in value:
                info_parts.append(value["response"])
            elif isinstance(value, str):
                info_parts.append(value)
        
        state["final_response"] = fallback_response + "\n".join(info_parts)
    
    else:
        # Create comprehensive response from all retrieved information
        if len(state["retrieved_info"]) == 1:
            # Single tool response
            key, value = list(state["retrieved_info"].items())[0]
            if isinstance(value, dict) and "response" in value:
                state["final_response"] = value["response"]
            else:
                state["final_response"] = str(value)
        
        else:
            # Multiple tool responses - consolidate
            consolidation_prompt = f"""
            The user asked: "{state['user_input']}"
            
            I have gathered information from multiple sources. Please create a comprehensive, 
            well-structured response that consolidates this information:
            
            """
            
            for key, value in state["retrieved_info"].items():
                if isinstance(value, dict) and "response" in value:
                    consolidation_prompt += f"\n{key.upper()}:\n{value['response']}\n"
                elif isinstance(value, str):
                    consolidation_prompt += f"\n{key.upper()}:\n{value}\n"
            
            consolidation_prompt += """
            
            Please provide a clear, comprehensive response that addresses the user's question 
            using all relevant information above. Structure it in a user-friendly way.
            """
            
            try:
                consolidated = bedrock_llm.invoke(consolidation_prompt).content
                state["final_response"] = consolidated
            except Exception as e:
                # Fallback to simple concatenation
                response_parts = []
                for key, value in state["retrieved_info"].items():
                    if isinstance(value, dict) and "response" in value:
                        response_parts.append(f"**{key.title()}**: {value['response']}")
                    elif isinstance(value, str):
                        response_parts.append(f"**{key.title()}**: {value}")
                
                state["final_response"] = "\n\n".join(response_parts)
    
    # Add to conversation history
    state["conversation_history"].append({
        "timestamp": datetime.now().isoformat(),
        "user_input": state["user_input"],
        "response": state["final_response"],
        "tools_used": state["tool_calls"]
    })
    
    state["current_step"] = "complete"
    return state

print("✅ Response generation implemented!")

# Graph Construction

In [ ]:
def start_node(state: AgentState) -> AgentState:
    """Initialize the conversation state"""
    if not state.get("conversation_history"):
        state["conversation_history"] = []
    
    state["messages"] = [f"Processing user input: {state['user_input']}"]
    state["current_step"] = "started"
    state["tool_calls"] = []
    state["retrieved_info"] = {}
    state["needs_more_tools"] = False
    state["final_response"] = ""
    
    return state

# Create the state graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("start", start_node)
workflow.add_node("router", router_node)
workflow.add_node("judge", judge_node)
workflow.add_node("create_response", create_final_response)

# Add edges
workflow.add_edge(START, "start")
workflow.add_edge("start", "router")
workflow.add_edge("router", "judge")
workflow.add_edge("judge", "create_response")
workflow.add_edge("create_response", END)

# Set up memory
memory = MemorySaver()

# Compile the graph
app = workflow.compile(checkpointer=memory)

print("✅ LangGraph workflow created successfully!")

# Graph Visualization

In [ ]:
try:
    from IPython.display import Image, display
    import base64
    from io import BytesIO
    
    # Get the graph visualization
    graph_image = app.get_graph().draw_mermaid_png()
    
    # Display the graph
    display(Image(graph_image))
    print("✅ Graph visualization displayed!")
    
except Exception as e:
    print(f"⚠️ Could not display graph visualization: {e}")
    print("Graph structure:")
    print("START -> start -> router -> judge -> create_response -> END")

# Chat Interface

In [ ]:
class ChatInterface:
    """Interactive chat interface for the agent"""
    
    def __init__(self, app, session_id="default_session"):
        self.app = app
        self.session_id = session_id
        self.config = {"configurable": {"thread_id": session_id}}
    
    def chat(self, user_input: str) -> str:
        """Process user input and return response"""
        
        # Create initial state
        initial_state = {
            "user_input": user_input,
            "messages": [],
            "current_step": "",
            "tool_calls": [],
            "retrieved_info": {},
            "final_response": "",
            "needs_more_tools": False,
            "conversation_history": []
        }
        
        try:
            # Run the workflow
            result = self.app.invoke(initial_state, config=self.config)
            return result["final_response"]
            
        except Exception as e:
            return f"Sorry, I encountered an error: {str(e)}"
    
    def get_conversation_history(self):
        """Retrieve conversation history from memory"""
        try:
            # Get the latest state from memory
            state = self.app.get_state(config=self.config)
            if state and state.values and "conversation_history" in state.values:
                return state.values["conversation_history"]
            return []
        except:
            return []
    
    def clear_history(self):
        """Clear conversation history"""
        try:
            # This would clear the thread - implementation depends on MemorySaver specifics
            self.session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            self.config = {"configurable": {"thread_id": self.session_id}}
            print("✅ Conversation history cleared!")
        except Exception as e:
            print(f"⚠️ Error clearing history: {e}")

# Initialize chat interface
chat_interface = ChatInterface(app)
print("✅ Chat interface ready!")

# Example Usage and Testing

In [ ]:
def test_chatbot():
    """Test the chatbot with various queries"""
    
    test_queries = [
        "Hi there!",
        "What changes are currently in progress?",
        "Tell me about server CI001",
        "Any critical incidents?",
        "Show me information about database changes and any related incidents",
        "What is artificial intelligence?"
    ]
    
    print("🧪 Testing Chatbot with Sample Queries")
    print("=" * 50)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n{i}. User: {query}")
        print("-" * 30)
        
        response = chat_interface.chat(query)
        print(f"Bot: {response}")
        
        print("-" * 30)

# Run tests
test_chatbot()

# Interactive Chat Loop

In [ ]:
def interactive_chat():
    """Interactive chat loop for real-time conversation"""
    
    print("\n🤖 IT Service Management Assistant")
    print("=" * 50)
    print("Type 'quit', 'exit', or 'bye' to end the conversation")
    print("Type 'history' to see conversation history")
    print("Type 'clear' to clear conversation history")
    print("=" * 50)
    
    while True:
        try:
            user_input = input("\n👤 You: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("👋 Goodbye! Have a great day!")
                break
            
            elif user_input.lower() == 'history':
                history = chat_interface.get_conversation_history()
                if history:
                    print("\n📚 Conversation History:")
                    for i, entry in enumerate(history[-5:], 1):  # Show last 5
                        print(f"{i}. {entry['timestamp'][:19]}")
                        print(f"   You: {entry['user_input']}")
                        print(f"   Bot: {entry['response'][:100]}...")
                        print(f"   Tools used: {', '.join(entry['tools_used'])}")
                else:
                    print("No conversation history available.")
                continue
            
            elif user_input.lower() == 'clear':
                chat_interface.clear_history()
                continue
            
            elif not user_input:
                continue
            
            print("\n🤖 Assistant: ", end="")
            response = chat_interface.chat(user_input)
            print(response)
            
        except KeyboardInterrupt:
            print("\n👋 Chat interrupted. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

# Start interactive chat
print("""
🎉 IT Service Management Assistant Setup Complete!

📋 SUMMARY:
- ✅ AWS Bedrock integration with Claude 3.5 Sonnet
- ✅ FAISS vector databases for Change, CMDB, and Incident data
- ✅ Self-query retrievers for intelligent information extraction
- ✅ LangGraph workflow with router, tools, and judge
- ✅ Memory persistence for conversation history
- ✅ Multi-tool capability for complex queries

🚀 USAGE:
1. Use interactive_chat() for real-time conversation
2. Use chat_interface.chat("your question") for single queries
3. Use test_chatbot() to run predefined tests

📝 AVAILABLE COMMANDS IN INTERACTIVE MODE:
- 'history' - View conversation history
- 'clear' - Clear conversation history  
- 'quit'/'exit'/'bye' - End conversation

🔧 CUSTOMIZATION:
- Replace sample Excel files with your actual data files
- Modify the create_vector_store_from_excel() calls with your file paths
- Adjust metadata fields and descriptions for your specific use case
- Customize the router logic for your specific tool requirements

📊 VECTOR STORES LOCATION: ./vector_stores/
💾 CONVERSATION MEMORY: Persisted automatically via MemorySaver

Happy chatting! 🎯
""")
print("\n" + "="*60)
print("Ready to start interactive chat!")
print("Run: interactive_chat()")
print("="*60)